# In this notebook several previously created models will be tested, namely:

#### Firstly, some models were trained on data without additional filtration

* Model trained on original WikiLarge data to perform task for English
* Model trained only on the translated to Russian data.
* Model trained firstly on the original data and then on the translated corpus
* Bonus Model trained on pairs: original english - simplified russian sentence. So, it learns both translate and simplify at the same time.

### Then, as the results were not satisfying, an addsitional filtration and data was added

* Model trained on filtered original WikiLarge data to perform task for English
* Model trained only on the filtere translated to Russian data.
* Model trained firstly on the filtered original data and then on the filtered translated corpus
* Model trained on russian corpus Paraphraser and the filtered translated corpus
* Model trained only on russian corpus Paraphraser

All the models will be tested on the dev part of the dataset presented in RuSimpleSentEval competition. For the two of the models the russian dataset firstly will be translated into English.

## Necessary libraries

In [ ]:
! pip install googletrans
! pip install transformers
! pip install laserembeddings
! pip install sentence_transformers
! pip install language_tool_python
! pip install textstat

In [2]:
! python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import torch
import language_tool_python
import re
import textstat
from laserembeddings import Laser
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import MarianMTModel, MarianTokenizer
from transformers.hf_api import HfApi
from googletrans import Translator
from tqdm import tqdm_notebook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

## Loading data...

In [ ]:
! mkdir data
! gdown https://drive.google.com/uc?id=1bJo8TagTGKa0uyppQRqsHrKHyYO5tcZc
! gdown https://drive.google.com/uc?id=11lqipq6ggrgCk8bVxQ4-uuPVMCKN5ebU
! gdown https://drive.google.com/uc?id=1dB3X-Wx8qU_5nDG_pxAmLvo5H_sgnHrE
! gdown https://drive.google.com/uc?id=1IVz3XC8Rm7hQCyx3xCcABjhaENrKABvF

In [5]:
test_data = pd.read_csv('/content/test_baselines.csv', sep='\t')
test_data['INPUT:source'] = test_data['INPUT:source'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data['OUTPUT:output'] = test_data['OUTPUT:output'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data.to_csv('test_data_new.csv', index=False, sep='\t')

In [6]:
data_train = pd.read_csv('/content/wiki_train_cleaned_translated_sd.csv')
data_dev = pd.read_csv('/content/wiki_dev_cleaned_translated_sd.csv')
data_test  = pd.read_csv('/content/test_data_new.csv',  sep='\t')

In [7]:
data_test.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,cos_sim_ref,cos_sim_tr_bs,cos_sim_gpt_bs
1141,3356,"Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу над романом в стихах, который был его «спутником верным» на протяжении восьми лет жизни.","Здесь он написал ""Письмо Онегина"" и этим закончил поэму, которую писал 8 лет.","Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу над романом в стихах,.","Здесь он пишет «Письмо Онегина», тем самым окончательно завершая работу о романе в 1843 году.",0.790754,0.844982,0.838458
314,1105,"В настоящее время не существует лекарств, способных обратить или хотя бы замедлить развитие болезни Альцгеймера.","Сейчас нет лекарств, которые могли хотя бы затормозить развитие болезни Альцгеймера.","В настоящее время не существует лекарств, способных обратить или.","В настоящее время не существует лекарств, способных обратить или хотя бы замедлить развитие болезни.",0.988584,0.985970,0.994800
1227,3610,Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов и должен был стать основным инструментом мирного урегулирования.,Имперский камерный суд теперь составляют католики и протестанты.,Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов.,"Имперский камерный суд состоял теперь в равной пропорции из католиков и протестантов, которые были равны между собой как по численности населения (апрель 1797 года), так даже для их веры.",0.951733,0.977822,0.750965
351,1181,В основе заболевания лежит ускорение инактивации инсулина или же специфическое разрушение рецепторов инсулина на мембранах инсулинзависимых клеток.,Причина заболевания - снижение инсулина или разрешение рецептеров инсулина на клетки;,В основе заболевания лежит ускорение инактивации инсулина или же специфическое.,В основе заболевания лежит ускорение инактивации инсулина или же специфическое повышение иммунитета в крови.,0.957886,0.966153,0.975032
914,2698,"Далее, азербайджанские войска начали наступление по нескольким направлениям с целью подчинить над армянскими сёлами Нагорного Карабаха.",войска азербайджана начали наступление с целью захвата Нагорного Карабаха,"Далее, азербайджанские войска начали наступление по нескольким направлениям с.","Далее, азербайджанские войска начали наступление по нескольким направлениям с целью подчинить над армянской армией Азербайджан и его армию в конечном итоге.",0.978669,0.981258,0.928678


## Setting the environment for Fairseq mBart

In [ ]:
! wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
! tar -xzvf /content/mbart.cc25.v2.tar.gz
! apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

In [8]:
! git clone https://github.com/google/sentencepiece.git 
%cd sentencepiece
! mkdir build

Cloning into 'sentencepiece'...
remote: Enumerating objects: 3690, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 3690 (delta 2), reused 1 (delta 0), pack-reused 3680
Receiving objects: 100% (3690/3690), 28.58 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (2585/2585), done.
/content/sentencepiece


In [ ]:
%cd build
!cmake ..
!make
!make install
!ldconfig -v

In [10]:
%cd /content

/content


In [ ]:
! git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
! python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH


In [ ]:
! git clone https://github.com/feralvam/easse
! git clone https://github.com/Andoree/sent_simplification.git
%cp /content/sent_simplification/sari.py /content/easse/easse

%cd easse
! pip install .

In [13]:
%cd /content

/content


In [1]:
! mkdir prepared_data
! mkdir preds

mkdir: cannot create directory ‘prepared_data’: File exists
mkdir: cannot create directory ‘preds’: File exists


## Translation to English

We have to translate data to English to test the english models and also the model trained on english source and russian target sentences

In [7]:
data_test = data_test[['Unnamed: 0', 'INPUT:source','OUTPUT:output', 'trunctuation_bs', 'gpt_bs']]

In [8]:
# helper functions

def batch_generator(
        list_of_sentences,
        size=64
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def translate_to_eng(model, tok, src, dst):

  translations_src = []
  for i in tqdm_notebook(batch_generator(src)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_src.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  translations_dst = []
  for i in tqdm_notebook(batch_generator(dst)):
    l = tok(i, return_tensors="pt", padding=True).to(device)
    translated = model.generate(**l)
    translations_dst.extend([tok.decode(t, skip_special_tokens=True) for t in translated])

  return translations_src, translations_dst

In [ ]:
# import en-ru model from transformers
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
model.to(device)

data_test['INPUT:source'] = data_test['INPUT:source'].astype(str)
data_test['OUTPUT:output'] = data_test['OUTPUT:output'].astype(str)

In [ ]:
# test--------------------------------------------------
src, dst = translate_to_eng(model, tokenizer, list(data_test['INPUT:source'].values), list(data_test['OUTPUT:output'].values))
for i, obj  in enumerate(src):
  src[i] = src[i].replace('&gt;', '')
  #src[i] = re.sub(r'[&gt;{1,}]', " ", src[i])
  src[i] = src[i].replace('&lt;', '')
  src[i] = src[i].replace('()', '')
  src[i] = src[i].replace('&quot;', '') 
  src[i] = src[i].replace('&apos;', '')
for i, obj in enumerate(dst):
  dst[i] = dst[i].replace('&gt;', '')
  dst[i] = dst[i].replace('&lt;', '')
  dst[i] = dst[i].replace('()', '')
  dst[i] = dst[i].replace('&quot;', '')
  dst[i] = dst[i].replace('&apos;', '')
data_test['src'] = src
data_test['dst'] = dst
data_test.to_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

In [13]:
data_test.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,src,dst
1021,3120,"Если нельзя установить, какая фигура была тронута первой, считается, что это фигура игрока, за которым очередь хода, а не его партнёра.","Если неизвестно, какая фигура была тронута первой, считается, что это фигура того игрока, которой должен ходить.","Если нельзя установить, какая фигура была тронута первой, считается, что это фигура.","Если нельзя установить, какая фигура была тронута первой, считается, что это фигура «неправильная».","If it is not possible to determine which figure was touched by the first one, it is believed to be the figure of the player behind whom the turn of the move is not his partner.","If it is not known which figure was touched by the first one, it is believed to be the figure of the player that should walk."
2001,5984,"Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие в результате бомбардировки казармы, поэтому жил в землянках.",Гарнизон Кинбурга не смог до наступления зимы отстроить сгоревшие после бомбардировки казармы. Поэтому солдатам пришлось разместится в вырытых землянках.,Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие.,"Оставленный в Кинбурне гарнизон не успел до холодов восстановить сгоревшие постройки и деревни, а также несколько других деревень.","The garrison left in Kinbourne was not able to recover the burning of the barracks, so it lived in the ground.","The Kinburg Garnizon was unable until winter to rebuild the burning barracks after the bombing, so the soldiers had to sit in dug-out strawberries."
602,1873,Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной блокировки из-за частых захватов/освобождений блокировок оказался слишком медленным.,Вариант доступа к отдельным объектам оказался слишком медленным,Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной.,"Вариант интерпретатора с синхронизацией доступа к отдельным объектам вместо глобальной блокировки в сети позволяет получить доступ только из одной области, например.",The interpretation option of synchronizing access to individual objects instead of global blockage due to frequent captures/releases of blocks has been too slow.,The option of access to individual facilities proved to be too slow
19,71,"А по размаху политических репрессий против инакомыслящих хрущёвский режим стоит несоизмеримо ближе к Сталину, чем брежневский.","Политический режим Хрущёва больше похож на режим Сталина, чем режим Брежнева.",А по размаху политических репрессий против инакомыслящих хрущёвский режим.,"А по размаху политических репрессий против инакомыслящих хрущёвский СССР был третьим советским государством, состоявшим из пяти округов.",And by the scale of the political repression against the dissenting Kraushkov regime is disproportionately closer to Stalin than the Brezhnevian regime.,Chrushchev's political regime is more like Stalin's regime than Brezhnev's.
2581,7658,"Различные же теории заговоров создают атмосферу страха, слухи и предрассудки, подобные теории осуждаются научным сообществом.",Найчное сообщества и рассуждения о теориях и их влияния на разум .,"Различные же теории заговоров создают атмосферу страха, слухи и.","Различные же теории заговоров создают атмосферу страха, слухи и предрассудки, подобные тому что говорят.","Various conspiracy theories, however, create an atmosphere of fear, rumours and prejudice, such theories being condemned by the scientific community.",Scientific communities and discourse about theories and their influence on the mind.


In [ ]:
#data_test = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv', sep='\t')

## Inference stage

All the models will be evaluated according to the following aspects:

* SARI
* BLEU
* FKGL
* Grammar aspect

All the predictions were saved in advance. If there is a need to make a new prediction, mBART model can be loaded but it requires much RAM:(
   Also, it is necessary to make the following change in /content/fairseq/fairseq/tasks/translation_from_pretrained_bart.py:

```
def __init__(self, args, src_dict, tgt_dict):
        super().__init__(args, src_dict, tgt_dict)
        self.args = args                  # add this line !!!!!
        self.langs = args.langs.split(",")
        for d in [src_dict, tgt_dict]:
            for l in self.langs:
```


## Evaluation functions

BLEU

In [31]:
# function retrieved from easse code
import numpy as np
from typing import List
import sacrebleu
import easse.utils.preprocessing as utils_prep
from collections import OrderedDict


def corpus_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'exp',
    smooth_value: float = None,
    force: bool = True,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = False,
):

    sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    return sacrebleu.corpus_bleu(
        sys_sents,
        refs_sents,
        smooth_method,
        smooth_value,
        force,
        lowercase=False,
        tokenize='none',
        use_effective_order=use_effective_order,
    ).score


def sentence_bleu(
    sys_sent: str,
    ref_sents: List[str],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    return corpus_bleu(
        [sys_sent],
        [[ref] for ref in ref_sents],
        smooth_method,
        smooth_value,
        force=True,
        lowercase=lowercase,
        tokenizer=tokenizer,
        use_effective_order=use_effective_order,
    )


def corpus_averaged_sentence_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    scores = []
    for sys_sent, *ref_sents in zip(sys_sents, *refs_sents):
        scores.append(
            sentence_bleu(
                sys_sent,
                ref_sents,
                smooth_method,
                smooth_value,
                lowercase=lowercase,
                tokenizer=tokenizer,
                use_effective_order=use_effective_order,
            )
        )
    return np.mean(scores)


### Flesch Kincaid Grade Level

In [51]:
# https://github.com/infoculture/plainrussian
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from math import sqrt
import csv


from numpy import mean, arange


# Russian sounds and characters
RU_CONSONANTS_LOW = [u'к', u'п', u'с', u'т', u'ф', u'х', u'ц', u'ч', u'ш', u'щ']
RU_CONSONANTS_HIGH = [u'б', u'в', u'г', u'д', u'ж', u'з']
RU_CONSONANTS_SONOR = [u'л', u'м', u'н', u'р']
RU_CONSONANTS_YET = [u'й']

RU_CONSONANTS = RU_CONSONANTS_HIGH + RU_CONSONANTS_LOW + RU_CONSONANTS_SONOR + RU_CONSONANTS_YET
RU_VOWELS = [u'а', u'е', u'и', u'у', u'о', u'я', u'ё', u'э', u'ю', u'я', u'ы']
RU_MARKS = [u'ь', u'ъ']
SENTENCE_SPLITTERS = [u'.', u'?', u'!']
RU_LETTERS = RU_CONSONANTS + RU_MARKS + RU_VOWELS
SPACES = [u' ', u'\t']

# List of prepared texts

GRADE_TEXT = {
    1: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    2: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    3: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    4: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    5: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    6: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    7: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    8: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    9: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    10: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    11: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    12: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    13: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    14: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    15: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    16: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    17: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
}

POST_GRADE_TEXT_18_24 = u'Аспирантура, второе высшее образование, phD'


def calc_SMOG(n_psyl, n_sent):
    """Метрика SMOG для английского языка"""
    n = 1.0430 * sqrt((float(30.0) / n_sent) * n_psyl) + 3.1291
    return n

def calc_Gunning_fog(n_psyl, n_words, n_sent):
    """Метрика Gunning fog для английского языка"""
    n = 0.4 * ((float(n_words)/ n_sent) + 100 * (float(n_psyl) / n_words))
    return n

def calc_Dale_Chale(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для английского языка"""
    n = 0.1579 * (100.0 * n_psyl / n_words) + 0.0496 * (float(n_words) / n_sent)
    return n

def calc_Flesh_Kincaid(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для английского языка"""
    n = 206.835 - 1.015 * (float(n_words) / n_sent) - 84.6 * (float(n_syllabes) / n_words)
    return n


def calc_Flesh_Kincaid_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для русского языка"""
    n = 220.755 - 1.315 * (float(n_words) / n_sent) - 50.1 * (float(n_syllabes) / n_words)
    return n

def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.49 * (float(n_words) / n_sent) + 7.3 * (float(n_syllabes) / n_words) - 16.59
    return n



def calc_Flesh_Kincaid_Grade_rus_adapted(n_syllabes, n_words, n_sent, X, Y, Z):
    """Метрика Flesh Kincaid Grade для русского языка с параметрами"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    if n_words == 0 or n_sent == 0: return 0
    n = X * (float(n_words) / n_sent) + Y * (float(n_syllabes) / n_words) - Z
    return n


#X_GRADE = 0.186
#Y_GRADE = 7.21
#Z_GRADE = 15.443

# Flesh Kinkaid Grade константы. Подробнее http://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests
FLG_X_GRADE = 0.318
FLG_Y_GRADE = 14.2
FLG_Z_GRADE = 30.5

def calc_Flesh_Kincaid_Grade_rus_flex(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка с константными параметрами"""
    if n_words == 0 or n_sent == 0: return 0
    n = FLG_X_GRADE * (float(n_words) / n_sent) + FLG_Y_GRADE * (float(n_syllabes) / n_words) - FLG_Z_GRADE
    return n


# Coleman Liau константы. Подробнее http://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index

CLI_X_GRADE = 0.055
CLI_Y_GRADE = 0.35
CLI_Z_GRADE = 20.33


def calc_Coleman_Liau_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Coleman Liau для русского языка с адаптированными параметрами """
    if n_words == 0: return 0
    n = x * (n_letters * (100.0 / n_words)) - y * (n_sent * (100.0 / n_words)) - z
    return n

def calc_Coleman_Liau_index(n_letters, n_words, n_sent):
    """ Метрика Coleman Liau для русского языка с константными параметрами """
    if n_words == 0: return 0
    n = CLI_X_GRADE * (n_letters * (100.0 / n_words)) - CLI_Y_GRADE * (n_sent * (100.0 / n_words)) - CLI_Z_GRADE
    return n


# Константы SMOG Index http://en.wikipedia.org/wiki/SMOG
SMOG_X_GRADE = 1.1
SMOG_Y_GRADE = 64.6
SMOG_Z_GRADE = 0.05

def calc_SMOG_index(n_psyl, n_sent):
    """Метрика SMOG для русского языка с константными параментрами"""
    n = SMOG_X_GRADE * sqrt((float(SMOG_Y_GRADE) / n_sent) * n_psyl) + SMOG_Z_GRADE
    return n

def calc_SMOG_index_adapted(n_psyl, n_sent, x, y, z):
    """Метрика SMOG для русского языка адаптированная с коэффициентами"""
    n = x * sqrt((float(y) / n_sent) * n_psyl) + z
    return n

DC_X_GRADE = 0.552
DC_Y_GRADE = 0.273

def calc_Dale_Chale_index(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для русского языка с константным параметрами"""
    n = DC_X_GRADE * (100.0 * n_psyl / n_words) + DC_Y_GRADE * (float(n_words) / n_sent)
    return n


def calc_Dale_Chale_adapted(n_psyl, n_words, n_sent, x, y):
    """Метрика Dale Chale для русского языка с адаптированными параметрами"""
    n = x * (100.0 * n_psyl / n_words) + y * (float(n_words) / n_sent)
    return n

ARI_X_GRADE = 6.26
ARI_Y_GRADE = 0.2805
ARI_Z_GRADE = 31.04


def calc_ARI_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Automated Readability Index (ARI) для русского языка с адаптированными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = x * (float(n_letters) / n_words) + y * (float(n_words) / n_sent) - z
    return n

def calc_ARI_index(n_letters, n_words, n_sent):
    """ Метрика Automated Readability Index (ARI) для русского языка с константными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = ARI_X_GRADE * (float(n_letters) / n_words) + ARI_Y_GRADE * (float(n_words) / n_sent) - ARI_Z_GRADE
    return n


def load_words(filename):
    """Load words from filename"""
    words = []
    f = open(filename, 'r')
    for l in f:
        words.append(l.strip().decode('utf8'))
    f.close()
    return words

#FAM_WORDS = load_words('1norm50000.txt')

bad_chars = '(){}<>"\'!?,.:;'


def calc_text_metrics(filename, verbose=True):
    """Расчет метрик"""
    f = open(filename, 'r')
    text = f.read().decode('utf8')    
    f.close()
    return calc_readability_metrics(text, verbose)


# Number of syllabes for long words
COMPLEX_SYL_FACTOR = 4

def calc_readability_metrics(text, verbose=True):
    sentences = 0
    chars = 0
    spaces = 0
    letters = 0
    syllabes = 0
    words = 0
    complex_words = 0
    simple_words = 0
    wsyllabes = {}

    wordStart = False
    for l in text.splitlines():
        chars += len(l)
#        l = l.decode('utf8')
        for ch in l:
            if ch in SENTENCE_SPLITTERS:
                sentences += 1
            if ch in SPACES:
                spaces += 1

        for w in l.split():
            has_syl = False
            wsyl = 0
#            if len(w) > 1: words += 1
            for ch in w:
                if ch in RU_LETTERS:
                    letters += 1
                if ch in RU_VOWELS:
                    syllabes += 1
                    has_syl = True
                    wsyl += 1
            if wsyl > COMPLEX_SYL_FACTOR:
                complex_words += 1
            elif wsyl < COMPLEX_SYL_FACTOR+1 and wsyl > 0:
                simple_words += 1
            if has_syl:
                words += 1
                v = wsyllabes.get(str(wsyl), 0)
                wsyllabes[str(wsyl)] = v + 1
    metrics = {'c_share': float(complex_words) * 100 / words if words > 0 else 0,
               'avg_slen' : float(words) / sentences if sentences > 0 else 0,
               'avg_syl' : float(syllabes) / words if words > 0 else 0,
               'n_syllabes': syllabes,
               'n_words' : words,
               'n_sentences': sentences,
               'n_complex_words': complex_words,
               'n_simple_words' : simple_words,
               'chars': chars,
               'letters' : letters,
               'spaces' : spaces,
               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus_flex(syllabes, words, sentences),
               'index_cl_rus' : calc_Coleman_Liau_index(letters, words, sentences),
               'index_dc_rus' : calc_Dale_Chale_index(complex_words, words, sentences),
               'index_SMOG_rus' : calc_SMOG_index(complex_words, sentences),
               'index_ari_rus' : calc_ARI_index(letters, words, sentences),
#               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus(syllabes, words, sentences),
               'wsyllabes' : wsyllabes
    }
    del text
    return metrics

#### COSINE SIMILARITY

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

# shape should be [1, something (768, ex)]

# import numpy as np
# def cs(a, b):
#   return (a @ b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def calc_cos_sim(df, model,tok, x, y, column_name):
    Cos_sim= []
    for index, row in df.iterrows():
        
        # original
          sentence_A = tok.encode(row[x], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_A = sentence_A.to(device)
          output = model(sentence_A)
          sent_emb = output[-1][0]
          emb_source = sent_emb.mean(axis=1)
          emb_source = emb_source.cpu().detach().numpy()

          sentence_B = tok.encode(row[y], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_B = sentence_B.to(device)
          output = model(sentence_B)
          sent_emb = output[-1][0]
          emb_target= sent_emb.mean(axis=1)
          emb_target = emb_target.cpu().detach().numpy()

          cos_val = cosine_similarity(emb_source.reshape(emb_source.shape[0], -1), emb_target.reshape(emb_target.shape[0], -1))[0][0]
          Cos_sim.append(cos_val)
    df[column_name] = Cos_sim

GRAMMAR

In [60]:
def get_mistakes_summary(df_test, x):
    test = list(df_test[x].values)
    matches = []
    for i in test:
      matches.extend(tool.check(i))

    categories = set([i.category for i in matches])

    categories = {i:0 for i in categories}

    for i in matches:
      categories[i.category]+=1

    return categories

Add necessary predictions to the dataframe

In [30]:
with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_en'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_ru'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru_ru'] = lines


# filtered data---------------------------
with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_en_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['ru_ru_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru_filtered.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['en_ru_ru_filtered'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser2.hyp", 'r') as f:
  lines = [i.strip()+'.' for i in f.readlines()]
data_test['paraphraser'] = lines

with open("/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser_ru2_12epochs.hyp", 'r') as f:
  lines = [i.strip() for i in f.readlines()]
data_test['paraphraser_ru'] = lines


In [6]:
data_test.head()

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,en_en,en_ru,ru_ru,en_ru_ru,ru_ru_filtered,en_ru_ru_filtered,paraphraser,paraphraser_ru
0,3,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок.,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, который тогда назывался Рабоче-Крестьянской республикой.","On December 14 , 1944 , Ichka was renamed the Soviet worktown , after which the village council became the Soviet town .","14 декабря 1944 года рабочий город Ика был переименован в советский рабочий город, после чего деревня стала советским городом.","14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.","14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.,В Ичке рабочий посёлок переименован в рабочий посёлок Советский.,14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский.
1,4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси",1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение было изменено на форму головы.,Model 172A was released in 1960 .,Модель 172A была выпущена в 1960 году.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Модель 172A была выпущена в 1960 году.,В 1960 году была выпущена модель 172A.
2,5,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,В выпущенной в 1960 году модель имела изменения в хвостовом оперении и руле направления,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,"1960 году была выпущена модель 172A. Изменения: хвостовое оперение, крылья и все другие детали сделали его менее приспособленным к жизни.",Model 172A was released in 1960 .,Модель 172A была выпущена в 1960 году.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Изменения: хвостовое оперение и руль направления с обратной стреловидностью.,Модель 172A была выпущена в 1960 году.,В 1960 году была выпущена модель 172A.
3,6,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: в хвосте и руле направления с обратной стреловидностью и крепления для поплавкового шасси.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления.,1960 году была выпущена модель 172A. Изменения: хвостовое оперение было заменено н

## Evaluation of English model trained on data without filtering

### SARI

In [22]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "src" \
--trg_column "dst" \
--output_dir /content/prepared_data

1000
3406
3401
Overall number of references: 3401


In [23]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [24]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en.hyp -q

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'sari': 21.231, 'quality_estimation': {'Compression ratio': 0.935, 'Sentence splits': 1.034, 'Levenshtein similarity': 0.957, 'Exact copies': 0.419, 'Additions proportion': 0.019, 'Deletions proportion': 0.084, 'Lexical complexity score': 8.294}}


### BLEU

In [32]:
# make lists of src, dst
src_column, trg_column = "en_en", "dst"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

41.05267985962619

### Flesh-Kincaid Grade level

In [36]:
textstat.set_lang('en')

MEAN

In [30]:
a = data_test['src'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en

11.071814445096855

In [31]:
a = data_test['en_en'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en

11.808367586611897

WHOLE TEXT

In [33]:
a = ' '.join(list(data_test['src'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

10.7

In [34]:
a = ' '.join(list(data_test['en_en'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

11.1

### Cosine Similarity

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("roberta-base") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base", config=config)
model.to(device)

In [39]:
calc_cos_sim(data_test, model, tok, 'src', 'en_en', 'cos_sim_en_en')
data_test.cos_sim_en_en.mean()

0.9784409024862598

### Grammar

In [ ]:
tool = language_tool_python.LanguageTool('en')

In [64]:
errors = get_mistakes_summary(data_test, 'en_en')
errors

{'CASING': 38,
 'COLLOCATIONS': 2,
 'GRAMMAR': 23,
 'MISC': 1,
 'PUNCTUATION': 183,
 'REDUNDANCY': 28,
 'STYLE': 3,
 'TYPOGRAPHY': 8081,
 'TYPOS': 10}

 ## Evaluation of English model with filtering

### SARI

In [33]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_en_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_en_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [34]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_en_filtered.hyp -q

{'sari': 34.596, 'quality_estimation': {'Compression ratio': 0.597, 'Sentence splits': 0.987, 'Levenshtein similarity': 0.727, 'Exact copies': 0.015, 'Additions proportion': 0.009, 'Deletions proportion': 0.414, 'Lexical complexity score': 8.217}}


### BLEU

In [35]:
# make lists of src, dst
src_column, trg_column = "en_en_filtered", "dst"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

40.97846360104546

### FKLG

MEAN

In [37]:
a = data_test['en_en_filtered'].values
en_en = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
en_en


9.058573106283028

WHOLE DATA

In [38]:
a = ' '.join(list(data_test['en_en_filtered'].values))
en_en = textstat.flesch_kincaid_grade(a)
en_en

8.4

COSINE SIMILARITY

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("roberta-base") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base", config=config)
model.to(device)

In [45]:
calc_cos_sim(data_test, model, tok, 'src', 'en_en_filtered', 'cos_sim_en_en_filtered')
data_test.cos_sim_en_en_filtered.mean()

0.9042795725763981

### Grammar

In [65]:
errors = get_mistakes_summary(data_test, 'en_en_filtered')
errors

{'CASING': 14,
 'GRAMMAR': 43,
 'PUNCTUATION': 29,
 'REDUNDANCY': 10,
 'TYPOGRAPHY': 5028,
 'TYPOS': 15}

## Evaluation of Russian model trained on data without filtering 

### SARI

In [14]:
! rm -r /content/prepared_data
! mkdir /content/prepared_data

In [15]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [23]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction__ru_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [24]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction__ru_ru.hyp -q

{'sari': 16.499, 'quality_estimation': {'Compression ratio': 0.968, 'Sentence splits': 1.003, 'Levenshtein similarity': 0.979, 'Exact copies': 0.68, 'Additions proportion': 0.011, 'Deletions proportion': 0.045, 'Lexical complexity score': 10.675}}


### BLEU

In [79]:
# make lists of src, dst
src_column, trg_column = "ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.70629761889349

### Flesh-Kincaid Grade level

In [56]:
calc_readability_metrics(' '.join(list(data_test['INPUT:source'].values)))

{'avg_slen': 15.85661345496009,
 'avg_syl': 3.0165752808988766,
 'c_share': 16.981573033707864,
 'chars': 473430,
 'index_SMOG_rus': 14.557852197020122,
 'index_ari_rus': 17.71203679321743,
 'index_cl_rus': 16.38818426966293,
 'index_dc_rus': 13.702683787810848,
 'index_fk_rus': 17.377772067441356,
 'letters': 393678,
 'n_complex_words': 9446,
 'n_sentences': 3508,
 'n_simple_words': 46179,
 'n_syllabes': 167797,
 'n_words': 55625,
 'spaces': 60292,
 'wsyllabes': {'1': 9867,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12788,
  '3': 13723,
  '4': 9801,
  '5': 6023,
  '6': 2491,
  '7': 677,
  '8': 173,
  '9': 37}}

In [53]:
calc_readability_metrics(' '.join(list(data_test['OUTPUT:output'].values)))

{'avg_slen': 12.795306388526727,
 'avg_syl': 2.8380374974526186,
 'c_share': 12.78275932341553,
 'chars': 317164,
 'index_SMOG_rus': 11.356979245625993,
 'index_ari_rus': 13.911711320522606,
 'index_cl_rus': 13.275588954554713,
 'index_dc_rus': 10.54920179059317,
 'index_fk_rus': 13.869039895378684,
 'letters': 259382,
 'n_complex_words': 5018,
 'n_sentences': 3068,
 'n_simple_words': 34238,
 'n_syllabes': 111410,
 'n_words': 39256,
 'spaces': 43375,
 'wsyllabes': {'1': 7690,
  '10': 11,
  '11': 1,
  '12': 1,
  '14': 1,
  '15': 1,
  '2': 10187,
  '3': 9546,
  '4': 6815,
  '5': 3294,
  '6': 1274,
  '7': 336,
  '8': 71,
  '9': 28}}

In [52]:
calc_readability_metrics(' '.join(list(data_test.ru_ru.values)))

{'avg_slen': 15.412050256996002,
 'avg_syl': 3.01406258684898,
 'c_share': 16.843606988679525,
 'chars': 458434,
 'index_SMOG_rus': 14.294811353101062,
 'index_ari_rus': 17.531640673671966,
 'index_cl_rus': 16.275580568061812,
 'index_dc_rus': 13.505160777911009,
 'index_fk_rus': 17.200720714980243,
 'letters': 381506,
 'n_complex_words': 9091,
 'n_sentences': 3502,
 'n_simple_words': 44882,
 'n_syllabes': 162678,
 'n_words': 53973,
 'spaces': 58381,
 'wsyllabes': {'1': 9581,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12369,
  '3': 13339,
  '4': 9593,
  '5': 5826,
  '6': 2384,
  '7': 648,
  '8': 156,
  '9': 32}}

COSINE SIMILARITY

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased", config=config)
model.to(device)

In [63]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru', 'cos_sim_ru_ru')
data_test.cos_sim_ru_ru.mean()

0.9955931413422174

### Grammar

In [66]:
tool = language_tool_python.LanguageTool('ru')

In [67]:
errors = get_mistakes_summary(data_test, 'ru_ru')
errors

{'CASING': 8,
 'GRAMMAR': 22,
 'LOGIC': 27,
 'PUNCTUATION': 20,
 'STYLE': 14,
 'TYPOGRAPHY': 393,
 'TYPOS': 800}

## Evaluation of Russian model without filtering pretrained on English data

### SARI

In [59]:
# ! rm -r /content/prepared_data
# ! mkdir /content/prepared_data

In [60]:
# ! python /content/sent_simplification/refs_to_easse_format.py \
# --input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
# --output_dataset_name test_ref_data \
# --src_column "INPUT:source" \
# --trg_column "OUTPUT:output" \
# --output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [25]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [26]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru_ru.hyp -q

{'sari': 16.939, 'quality_estimation': {'Compression ratio': 0.963, 'Sentence splits': 1.002, 'Levenshtein similarity': 0.975, 'Exact copies': 0.652, 'Additions proportion': 0.013, 'Deletions proportion': 0.051, 'Lexical complexity score': 10.682}}


### BLEU

In [64]:
# make lists of src, dst
# src_column, trg_column = "en_ru_ru", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

34.749903130866464

In [80]:
# make lists of src, dst
src_column, trg_column = "en_ru_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.94343271922458

FKGL

In [65]:
calc_readability_metrics(' '.join(list(data_test.en_ru_ru.values)))

{'avg_slen': 15.32686822589846,
 'avg_syl': 3.014385142176567,
 'c_share': 16.906729194580915,
 'chars': 456511,
 'index_SMOG_rus': 14.281984238205567,
 'index_ari_rus': 17.53083943300245,
 'index_cl_rus': 16.283248101831177,
 'index_dc_rus': 13.516749541078948,
 'index_fk_rus': 17.17821311474296,
 'letters': 380029,
 'n_complex_words': 9085,
 'n_sentences': 3506,
 'n_simple_words': 44651,
 'n_syllabes': 161981,
 'n_words': 53736,
 'spaces': 58090,
 'wsyllabes': {'1': 9523,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12377,
  '3': 13254,
  '4': 9497,
  '5': 5829,
  '6': 2373,
  '7': 636,
  '8': 163,
  '9': 39}}

COSINE SIMILARITY

In [66]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru', 'cos_sim_en_ru_ru')
data_test.cos_sim_en_ru_ru.mean()

0.9955931413422174

### Grammar

In [71]:
errors = get_mistakes_summary(data_test, 'en_ru_ru')
errors

{'CASING': 17,
 'GRAMMAR': 20,
 'LOGIC': 27,
 'PUNCTUATION': 21,
 'STYLE': 14,
 'TYPOGRAPHY': 398,
 'TYPOS': 800}

# Evaluation of Russian model trained on filtered data

### SARI

In [42]:
# ! rm -r /content/prepared_data
# ! mkdir /content/prepared_data

# ! python /content/sent_simplification/refs_to_easse_format.py \
# --input_path /content/drive/MyDrive/MT_sentence_simpl/wiki_test_dev_eng.csv \
# --output_dataset_name test_ref_data \
# --src_column "INPUT:source" \
# --trg_column "OUTPUT:output" \
# --output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [27]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_ru_ru_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_ru_ru_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [28]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_ru_ru_filtered.hyp -q

{'sari': 32.31, 'quality_estimation': {'Compression ratio': 0.661, 'Sentence splits': 0.988, 'Levenshtein similarity': 0.778, 'Exact copies': 0.014, 'Additions proportion': 0.014, 'Deletions proportion': 0.361, 'Lexical complexity score': 10.721}}


### BLEU

In [68]:
# make lists of src, dst
# src_column, trg_column = "ru_ru_filtered", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [82]:
# make lists of src, dst
src_column, trg_column = "ru_ru_filtered", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

37.2531944122807

### FKLG

In [69]:
calc_readability_metrics(' '.join(list(data_test.ru_ru_filtered.values)))

{'avg_slen': 10.573348918760958,
 'avg_syl': 3.032502349234426,
 'c_share': 16.726549112818528,
 'chars': 310421,
 'index_SMOG_rus': 11.807592421245358,
 'index_ari_rus': 16.532692980412904,
 'index_cl_rus': 15.551128738046543,
 'index_dc_rus': 12.11957936509757,
 'index_fk_rus': 15.92385831529483,
 'letters': 257822,
 'n_complex_words': 6052,
 'n_sentences': 3422,
 'n_simple_words': 30130,
 'n_syllabes': 109722,
 'n_words': 36182,
 'spaces': 38974,
 'wsyllabes': {'1': 5850,
  '10': 18,
  '11': 6,
  '12': 1,
  '2': 8631,
  '3': 9238,
  '4': 6411,
  '5': 3882,
  '6': 1585,
  '7': 428,
  '8': 110,
  '9': 22}}

COSINE SIMILARITY

In [70]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'ru_ru_filtered', 'cos_sim_ru_ru_filtered')
data_test.cos_sim_ru_ru_filtered.mean()

0.9557879034449476

### Grammar

In [72]:
errors = get_mistakes_summary(data_test, 'ru_ru_filtered')
errors

{'CASING': 32,
 'GRAMMAR': 31,
 'LOGIC': 27,
 'PUNCTUATION': 22,
 'STYLE': 2,
 'TYPOGRAPHY': 197,
 'TYPOS': 571}

# Evaluation of Russian model trained on filtered data and pretrained on english data

### SARI

In [29]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru_ru_filtered.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru_ru_filtered.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [30]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru_ru_filtered.hyp -q

{'sari': 32.234, 'quality_estimation': {'Compression ratio': 0.658, 'Sentence splits': 0.988, 'Levenshtein similarity': 0.775, 'Exact copies': 0.018, 'Additions proportion': 0.017, 'Deletions proportion': 0.367, 'Lexical complexity score': 10.727}}


### BLEU

In [72]:
# make lists of src, dst
# src_column, trg_column = "en_ru_ru_filtered", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [83]:
# make lists of src, dst
src_column, trg_column = "en_ru_ru_filtered", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

36.79295860256343

### FKLG

In [73]:
calc_readability_metrics(' '.join(list(data_test.en_ru_ru_filtered.values)))

{'avg_slen': 10.63018096906013,
 'avg_syl': 3.0381394327136935,
 'c_share': 16.853840028556522,
 'chars': 312398,
 'index_SMOG_rus': 11.883922121789904,
 'index_ari_rus': 16.546607739909724,
 'index_cl_rus': 15.56704549822895,
 'index_dc_rus': 12.205359100316617,
 'index_fk_rus': 16.021977492695562,
 'letters': 259499,
 'n_complex_words': 6138,
 'n_sentences': 3426,
 'n_simple_words': 30281,
 'n_syllabes': 110646,
 'n_words': 36419,
 'spaces': 39266,
 'wsyllabes': {'1': 5932,
  '10': 25,
  '11': 6,
  '12': 1,
  '2': 8604,
  '3': 9235,
  '4': 6510,
  '5': 3952,
  '6': 1564,
  '7': 450,
  '8': 121,
  '9': 19}}

In [74]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru_ru_filtered', 'cos_sim_en_ru_ru_filtered')
data_test.cos_sim_en_ru_ru_filtered.mean()

0.9578453072041045

### Grammar

In [69]:
errors = get_mistakes_summary(data_test, 'en_ru_ru_filtered')
errors

{'CASING': 35,
 'GRAMMAR': 33,
 'LOGIC': 27,
 'PUNCTUATION': 11,
 'STYLE': 1,
 'TYPOGRAPHY': 218,
 'TYPOS': 578}

# Evaluation of Russian model trained on filtered data and pretrained on paraphraser

### SARI

In [31]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser_ru2_12epochs.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_paraphraser_ru2_12epochs.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [33]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_paraphraser_ru2_12epochs.hyp -q

{'sari': 31.812, 'quality_estimation': {'Compression ratio': 0.694, 'Sentence splits': 0.987, 'Levenshtein similarity': 0.798, 'Exact copies': 0.051, 'Additions proportion': 0.014, 'Deletions proportion': 0.332, 'Lexical complexity score': 10.707}}


### BLEU

In [77]:
# make lists of src, dst
# src_column, trg_column = "paraphraser_ru", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

53.16967153331756

In [84]:
# make lists of src, dst
src_column, trg_column = "paraphraser_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

37.98414836939204

### FKLG

In [76]:
calc_readability_metrics(' '.join(list(data_test.paraphraser_ru.values)))

{'avg_slen': 11.104956268221574,
 'avg_syl': 3.0341822000525074,
 'c_share': 16.789183512733,
 'chars': 325728,
 'index_SMOG_rus': 12.122081494236964,
 'index_ari_rus': 16.62973834297624,
 'index_cl_rus': 15.66384352848516,
 'index_dc_rus': 12.299282360253105,
 'index_fk_rus': 16.116763334040066,
 'letters': 271101,
 'n_complex_words': 6395,
 'n_sentences': 3430,
 'n_simple_words': 31695,
 'n_syllabes': 115572,
 'n_words': 38090,
 'spaces': 41033,
 'wsyllabes': {'1': 6297,
  '10': 23,
  '11': 6,
  '12': 1,
  '2': 8982,
  '3': 9561,
  '4': 6855,
  '5': 4047,
  '6': 1717,
  '7': 464,
  '8': 118,
  '9': 19}}

In [79]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'paraphraser_ru', 'cos_sim_paraphraser_ru')
data_test.cos_sim_paraphraser_ru.mean()

0.9564811277410527

### Grammar

In [73]:
errors = get_mistakes_summary(data_test, 'paraphraser_ru')
errors

{'CASING': 22,
 'GRAMMAR': 39,
 'LOGIC': 27,
 'PUNCTUATION': 29,
 'TYPOGRAPHY': 183,
 'TYPOS': 605}

# Evaluation of Russian model trained on Paraphraser

### SARI

In [34]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_paraphraser.hyp', 'r') as f:
  sentences = [i.strip()+'.' for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_paraphraser.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [35]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_paraphraser.hyp -q

{'sari': 35.149, 'quality_estimation': {'Compression ratio': 0.624, 'Sentence splits': 0.985, 'Levenshtein similarity': 0.711, 'Exact copies': 0.026, 'Additions proportion': 0.053, 'Deletions proportion': 0.441, 'Lexical complexity score': 10.732}}


### BLEU

In [ ]:
# make lists of src, dst
# src_column, trg_column = "paraphraser", "OUTPUT:output"
# data_dict = OrderedDict()
# for ind, row in data_test.iterrows():
#         source_sentence = row[src_column]
#         reference_sentence = row[trg_column]
#         if data_dict.get(source_sentence) is None:
#             data_dict[source_sentence] = []
#         data_dict[source_sentence].append(reference_sentence)
# max_num_reference_sentences = max([len(x) for x in data_dict.values()])
# src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
# corpus_averaged_sentence_bleu(src, dst)

In [85]:
# make lists of src, dst
src_column, trg_column = "paraphraser", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs)

30.314500674726755

### FKLG

In [89]:
calc_readability_metrics(' '.join(list(data_test.paraphraser.values)))

{'avg_slen': 9.505404615834063,
 'avg_syl': 3.06082306297446,
 'c_share': 16.977594738297938,
 'chars': 281270,
 'index_SMOG_rus': 11.28135952373477,
 'index_ari_rus': 16.624886027319754,
 'index_cl_rus': 15.523413037465048,
 'index_dc_rus': 11.966607755663162,
 'index_fk_rus': 15.986406162072562,
 'letters': 233885,
 'n_complex_words': 5524,
 'n_sentences': 3423,
 'n_simple_words': 27013,
 'n_syllabes': 99590,
 'n_words': 32537,
 'spaces': 35374,
 'wsyllabes': {'1': 5184,
  '10': 18,
  '11': 1,
  '12': 5,
  '2': 7437,
  '3': 8443,
  '4': 5949,
  '5': 3489,
  '6': 1503,
  '7': 387,
  '8': 105,
  '9': 16}}

COSINE SIMILARITY

In [90]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'paraphraser', 'cos_sim_paraphraser')
data_test.cos_sim_paraphraser.mean()

0.9374921821709667

### Grammar

In [74]:
errors = get_mistakes_summary(data_test, 'paraphraser')
errors

{'CASING': 29,
 'GRAMMAR': 29,
 'LOGIC': 27,
 'MISC': 9,
 'PUNCTUATION': 30,
 'STYLE': 1,
 'TYPOGRAPHY': 158,
 'TYPOS': 692}

# BONUS EN-RU MODEL

### SARI

In [43]:
with open('/content/drive/MyDrive/MT_sentence_simpl/predictions/model_prediction_en_ru.hyp', 'r') as f:
  sentences = [i.strip() for i in f.readlines()]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction_en_ru.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [44]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction_en_ru.hyp -q

{'sari': 37.865, 'quality_estimation': {'Compression ratio': 0.875, 'Sentence splits': 1.002, 'Levenshtein similarity': 0.665, 'Exact copies': 0.0, 'Additions proportion': 0.4, 'Deletions proportion': 0.497, 'Lexical complexity score': 10.398}}


### BLEU

In [59]:
# make lists of src, dst
src_column, trg_column = "en_ru", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in data_test.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)
  
corpus_bleu(src, refs) ###???

13.214882473186714

FKLG

In [ ]:
data_test.head()

In [53]:
calc_readability_metrics(' '.join(list(data_test['INPUT:source'].values)))

{'avg_slen': 15.85661345496009,
 'avg_syl': 3.0165752808988766,
 'c_share': 16.981573033707864,
 'chars': 473430,
 'index_SMOG_rus': 14.557852197020122,
 'index_ari_rus': 17.71203679321743,
 'index_cl_rus': 16.38818426966293,
 'index_dc_rus': 13.702683787810848,
 'index_fk_rus': 17.377772067441356,
 'letters': 393678,
 'n_complex_words': 9446,
 'n_sentences': 3508,
 'n_simple_words': 46179,
 'n_syllabes': 167797,
 'n_words': 55625,
 'spaces': 60292,
 'wsyllabes': {'1': 9867,
  '10': 35,
  '11': 6,
  '12': 4,
  '2': 12788,
  '3': 13723,
  '4': 9801,
  '5': 6023,
  '6': 2491,
  '7': 677,
  '8': 173,
  '9': 37}}

In [54]:
calc_readability_metrics(' '.join(list(data_test['OUTPUT:output'].values)))

{'avg_slen': 12.795306388526727,
 'avg_syl': 2.8380374974526186,
 'c_share': 12.78275932341553,
 'chars': 317164,
 'index_SMOG_rus': 11.356979245625993,
 'index_ari_rus': 13.911711320522606,
 'index_cl_rus': 13.275588954554713,
 'index_dc_rus': 10.54920179059317,
 'index_fk_rus': 13.869039895378684,
 'letters': 259382,
 'n_complex_words': 5018,
 'n_sentences': 3068,
 'n_simple_words': 34238,
 'n_syllabes': 111410,
 'n_words': 39256,
 'spaces': 43375,
 'wsyllabes': {'1': 7690,
  '10': 11,
  '11': 1,
  '12': 1,
  '14': 1,
  '15': 1,
  '2': 10187,
  '3': 9546,
  '4': 6815,
  '5': 3294,
  '6': 1274,
  '7': 336,
  '8': 71,
  '9': 28}}

In [52]:
calc_readability_metrics(' '.join(list(data_test.en_ru.values)))

{'avg_slen': 13.871391826236067,
 'avg_syl': 2.8955208505027197,
 'c_share': 14.082330641173561,
 'chars': 411959,
 'index_SMOG_rus': 12.406806113663269,
 'index_ari_rus': 15.059364916077413,
 'index_cl_rus': 14.23091354870612,
 'index_dc_rus': 11.560336482490253,
 'index_fk_rus': 15.027498677881688,
 'letters': 327257,
 'n_complex_words': 6835,
 'n_sentences': 3499,
 'n_simple_words': 41701,
 'n_syllabes': 140537,
 'n_words': 48536,
 'spaces': 54374,
 'wsyllabes': {'1': 8966,
  '10': 4,
  '2': 12415,
  '3': 11739,
  '4': 8581,
  '5': 4441,
  '6': 1876,
  '7': 428,
  '8': 71,
  '9': 15}}

COSINE SIMILARITY

In [56]:
calc_cos_sim(data_test, model, tok, 'INPUT:source', 'en_ru', 'cos_sim_en_ru')
data_test.cos_sim_en_ru.mean()

0.9811948919324265

### Grammar

In [75]:
errors = get_mistakes_summary(data_test, 'en_ru')
errors

{'CASING': 57,
 'GRAMMAR': 163,
 'LOGIC': 12,
 'MISC': 131,
 'PUNCTUATION': 48,
 'STYLE': 27,
 'TYPOGRAPHY': 230,
 'TYPOS': 1073}